In [1]:
%load_ext autoreload
%autoreload 2
import os
from dimcat import (
    Corpus,  
    Pipeline,
    IsAnnotatedFilter,
    CorpusGrouper, 
    PieceGrouper, 
    ModeGrouper, 
    ChordSymbolBigrams, 
    ChordSymbolUnigrams,
    LocalKeySlicer,
)
from dimcat import __version__ as dimcat_version
from ms3 import __version__ as ms3_version
from git import Repo
import pandas as pd
pd.set_option("display.max_columns", 100)

In [2]:
corpus_path = "~/romantic_piano_corpus"

repo = Repo(corpus_path)
print(f"{os.path.basename(corpus_path)} @ {repo.commit().hexsha[:7]}")
print(f"dimcat version {dimcat_version}")
print(f"ms3 version {ms3_version}")

romantic_piano_corpus @ 025834b
dimcat version 0.2.0.post1.dev35+g9830b67
ms3 version 0.5.3.post0.dev88+g764fcd4.dirty


In [3]:
Repo.working_dir

In [4]:
corpus = Corpus(directory=corpus_path)
corpus.data

1154 files.
KEY                       -> EXTENSIONS
---------------------------------------
beethoven_piano_sonatas   -> {'.mscx': 87, '.tsv': 239}
chopin_mazurkas           -> {'.mscx': 55, '.tsv': 166}
debussy_suite_bergamasque -> {'.mscx': 4, '.tsv': 13}
dvorak_silhouettes        -> {'.mscx': 12, '.tsv': 37}
grieg_lyrical_pieces      -> {'.mscx': 66, '.tsv': 199}
liszt_pelerinage          -> {'.mscx': 19, '.tsv': 77}
medtner_tales             -> {'.mscx': 19, '.tsv': 59}
schumann_kinderszenen     -> {'.mscx': 13, '.tsv': 40}
tchaikovsky_seasons       -> {'.mscx': 12, '.tsv': 37}

None of the 287 score files have been parsed.

All 867 tabular files have been parsed, 283 of them as Annotations object(s).
KEY                       -> ANNOTATION LAYERS
----------------------------------------------
beethoven_piano_sonatas   -> staff  voice  harmony_layer  color  
                          -> 2      1      0 (dcml)       default    20720
                          ->        2      0 (dcml

# Metadata

In [5]:
all_metadata = corpus.data.metadata(from_tsv=True)
print(f"Concatenated 'metadata.tsv' files cover {len(all_metadata)} of the {len(corpus.data._score_ids())} scores.")
all_metadata.sample(10)

Concatenated 'metadata.tsv' files cover 287 of the 287 scores.


,,,rel_paths,fnames,last_mc,last_mn,length_qb,length_qb_unfolded,all_notes_qb,n_onsets,n_onset_positions,TimeSig,KeySig,label_count,annotated_key,annotators,reviewers,composer,workTitle,movementNumber,movementTitle,workNumber,poet,lyricist,arranger,copyright,creationDate,mscVersion,platform,source,translator,musescore,ambitus,PDF,Reviewers,annotator,comments,composed_end,composed_start,harmony_version,imslp,key,mode,originalFormat,pdf,score integrity,score_integrity,staff_1_ambitus,staff_1_instrument,staff_2_ambitus,staff_2_instrument,staff_3_ambitus,staff_3_instrument,staff_4_ambitus,staff_4_instrument,typesetter
key,i,metadata_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
schumann_kinderszenen,39,4,MS3,n05,19,16,36.50,NaN,154.25,375.0,144.0,1: 2/4,1: 2,48,D,"Tal Soker (2.1.1), John Heilig (2.3.0)","AN, JHei, JH",Robert Schumann,Kinderszenen Nos. 7 and 8,,NaN,NaN,NaN,NaN,NaN,NaN,2017-03-11,3.02,Microsoft Windows,http://musescore.com/user/22249306/scores/4778176,NaN,3.6.2,33-81 (A1-A5),NaN,NaN,NaN,measurcount differs from pdf � second house no...,1838,1838,2.3.0,NaN,NaN,NaN,NaN,https://imslp.org/wiki/Special:ReverseLookup/6...,NaN,Tom Schreyer,54-81 (F#3-A5),Piano,33-62 (A1-D4),Piano,NaN,NaN,NaN,NaN,NaN
dvorak_silhouettes,36,7,MS3,op08n08,57,57,NaN,NaN,NaN,NaN,NaN,1: 2/4,1: 2,100,b,"Adrian Nagel (2.1.1), Hanné Becker (2.3.0)",Adrian Nagel (2.1.1.),Antonín Dvořák,Silhouettes,8,Allegretto,op. 8,NaN,NaN,NaN,NaN,2019-04-04,3.02,Apple Macintosh,NaN,NaN,3.6.2,30-90 (F#1-F#6),NaN,NaN,NaN,NaN,1879,1870,2.3.0,NaN,NaN,NaN,xml,https://imslp.org/wiki/Special:ReverseLookup/5...,Tom Schreyer,NaN,49-90 (C#3-F#6),Piano,30-76 (F#1-E5),Piano,NaN,NaN,NaN,NaN,NaN
liszt_pelerinage,76,2,MS3,160.03_Pastorale,49,48,206.00,206.00,705.00,984.0,NaN,"12: 6/8, 25: 12/8, 35: 6/8",1: 4,200,E,"Adrian Nagel (2.1.1), Amelia Brey (2.3.0)","JH, AB, AN",Franz Liszt,Au bord d'une source,,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-26,3.02,Microsoft Windows,http://musescore.com/user/2749876/scores/4094931,NaN,3.6.2,40-87 (E2-D#6),https://imslp.org/wiki/Special:ReverseLookup/1...,NaN,NaN,NaN,1855,1848,2.3.0,NaN,NaN,NaN,mxl,NaN,NaN,Tom Schreyer,59-87 (B3-D#6),Piano,40-64 (E2-E4),Piano,NaN,NaN,NaN,NaN,https://musescore.com/user/2749876
debussy_suite_bergamasque,12,2,MS3,l075-03_suite_clair,72,72,324.00,324.00,1464.00,1559.0,NaN,NaN,"1: -5, 37: 4, 43: -5",150,Db,"Adrian Nagel (2.1.1), Amelia Brey (2.3.0)","AB, AN",Claude Debussy,Suite Bergamasque,,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-19,3.02,Microsoft Windows,http://musescore.com/score/890041,NaN,3.6.2,27-97 (Eb1-C#7),NaN,NaN,NaN,NaN,1905,1890,2.3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54-97 (Gb3-C#7),Piano,27-97 (Eb1-C#7),Piano,NaN,NaN,NaN,NaN,NaN
beethoven_piano_sonatas,238,81,MS3,30-3,211,203,673.50,814.50,2061.96,4118.0,NaN,"68: 2/4, 100: 9/8, 120: 2/2, 160: 3/4, 164: 9/...",1: 4,656,E,Adrian Nagel (2.3.0),Victor Zheng,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-29,3.02,Linux,NaN,NaN,3.6.2,30-95 (F#1-B6),NaN,Victor Zheng,NaN,NaN,1820,1820,2.3.0,NaN,NaN,NaN,xml,NaN,NaN,NaN,47-95 (B2-B6),NaN,30-84 (F#1-C6),NaN,NaN,NaN,NaN,NaN,NaN
dvorak_silhouettes,36,9,MS3,op08n10,58,58,NaN,NaN,NaN,NaN,NaN,1: 6/8,1: 1,104,G,"Adrian Nagel (2.1.1), Hanné Becker (2.3.0)",Adrian Nagel (2.1.1),Antonín Dvořák,Silhouettes,10,Allegretto grazioso,op. 8,NaN,NaN,NaN,NaN,2019-04-04,3.02,Apple Macintosh,NaN,NaN,3.6.2,35-93 (B1-A6),NaN,NaN,NaN,NaN,1879,1870,2.3.0,NaN,NaN,NaN,xml,https://imslp.org/wiki/Special:ReverseLookup/5...,Tom Schreyer,NaN,51-93 (D#3-A6),Piano,35-81 (B1-A5),Piano,NaN,NaN,NaN,NaN,NaN
liszt_pelerinage,76,18,MS3,162.03_Tarantella_da_Guillaume_Louis_Cottrau._...,481,479,1173.88,1173.88,3096.75,7726.0,NaN,"1: 6/8, 201: 2/4","1: -2, 201: -3, 261: 4, 267: -3, 336: 4, 345: 1",716,g,"Adrian Nagel (2.1.1), Amelia Brey (2.3.0)","JH, AB, AN",Gabriel Fauré,Nocturne No. 1,,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-26,3.02,Microsoft Windows,http://musescore.com/user/2749876/scores/5399463,NaN,3.6.2,26-92 (D1-Ab6),https://imslp.org/wiki/Special:ReverseLookup/1...,NaN,

In [6]:
print("VALUE COUNTS OF THE COLUMN 'annotators'")
all_metadata.annotators.value_counts()

VALUE COUNTS OF THE COLUMN 'annotators'


Adrian Nagel (2.1.1), John Heilig (2.3.0)                               74
Wendelin Bitzan (1.0.0), Adrian Nagel (2.2.0), Davor Krkljus (2.3.0)    49
Adrian Nagel (2.3.0)                                                    16
Adrian Nagel (2.1.1), Amelia Brey (2.3.0)                               15
Wendelin Bitzan (2.2.0), John Heilig (2.3.0)                            15
Tal Soker (2.1.1), John Heilig (2.3.0)                                  13
Lydia Carlisi (2.2.0), Adrian Nagel (2.3.0)                             11
Adrian Nagel                                                             9
Adrian Nagel (2.1.1), Hanné Becker (2.3.0)                               9
Adrian Nagel (2.2.0), Amelia Brey (2.3.0)                                6
Daniel Grote (2.2.0), Adrian Nagel (2.3.0)                               6
Adrian Nagel (2.2.0), Hanné Becker (2.3.0)                               5
Wendelin Bitzan                                                          5
Lydia Carlisi (2.2.0), Vi

In [7]:
print(f"Composition dates range from {all_metadata.composed_start.min()} ({all_metadata.loc[all_metadata.composed_start.idxmin(), 'fnames']}) "
      f"to {all_metadata.composed_end.max()} ({all_metadata.loc[all_metadata.composed_end.idxmax(), 'fnames']}).")

Composition dates range from 1794 (01-1) to 1925 (op08n01).


In [8]:
annotated = IsAnnotatedFilter().process_data(corpus)
print(f"Before: {len(corpus.indices[()])} IDs, after filtering: {len(annotated.indices[()])}")

Before: 287 IDs, after filtering: 264


**Choose here if you want to see stats for all or only for annotated scores.**

In [9]:
#selected = corpus
selected = annotated

# Notes

In [10]:
all_notes = selected.get_facet('notes')
all_notes.sample(20)

quarterbeats  \
corpus                  fname                                              interval                          
beethoven_piano_sonatas 32-1                                               [561.125, 561.375)       4489/8   
chopin_mazurkas         BI89-4op24-4                                       [32.0, 33.0)                 32   
medtner_tales           op26n03                                            [108.5, 109.0)            217/2   
beethoven_piano_sonatas 30-1                                               [14.75, 15.0)              59/4   
grieg_lyrical_pieces    op65n06                                            [100.0, 101.0)              100   
beethoven_piano_sonatas 07-3                                               [220.0, 221.0)              220   
tchaikovsky_seasons     op37a12                                            [235.0, 236.0)              235   
grieg_lyrical_pieces    op54n04                                            [247.5, 250.5)            495/2   
                        op71n04                                            [244.0, 248.0)              244   
dvorak_silhouettes      op08n05                                            [220.0, 222.0)              220   
schumann_kinderszenen   n09                                                [68.0, 69.0)                 68   
liszt_pelerinage        162.03_Tarantella_da_Guillaume_Louis_Cottrau._P... [722.0, 722.375)            722   
medtner_tales           op34n01                                            [445.0, 446.0)              445   
liszt_pelerinage        162.01_Gondoliera                                  [73.5, 74.5)              147/2   
beethoven_piano_sonatas 18-4                                               [169.0, 170.0)              169   
                        16-2                                               [483.625, 485.125)       3869/8   
                        01-1                                               [17.0, 18.5)                 17   
medtner_tales           op34n04                                            [85.0, 85.25)                85   
liszt_pelerinage        161.04_Sonetto_47_del_Petrarca                     [425.0, 425.5)              425   
beethoven_piano_sonatas 23-3                                               [234.0, 235.0)              234   

                                                                                               duration_qb  \
corpus                  fname                                              interval                          
beethoven_piano_sonatas 32-1                                               [561.125, 561.375)        0.250   
chopin_mazurkas         BI89-4op24-4                                       [32.0, 33.0)              1.000   
medtner_tales           op26n03                                            [108.5, 109.0)            0.500   
beethoven_piano_sonatas 30-1                                               [14.75, 15.0)             0.250   
grieg_lyrical_pieces    op65n06                                            [100.0, 101.0)            1.000   
beethoven_piano_sonatas 07-3                                               [220.0, 221.0)            1.000   
tchaikovsky_seasons     op37a12                                            [235.0, 236.0)            1.000   
grieg_lyrical_pieces    op54n04                                            [247.5, 250.5)            3.000   
                        op71n04                                            [244.0, 248.0)            4.000   
dvorak_silhouettes      op08n05                                            [220.0, 222.0)            2.000   
schumann_kinderszenen   n09                                                [68.0, 69.0)              1.000   
liszt_pelerinage        162.03_Tarantella_da_Guillaume_Louis_Cottrau._P... [722.0, 722.375)          0.375   
medtner_tales           op34n01                                            [445.0, 446.0)            1.000   
liszt_pelerinage       

In [11]:
print(f"{len(all_notes)} notes distributed over staves as follows:")
all_notes.staff.value_counts()

434220 notes distributed over staves as follows:


1    230221
2    200618
3      2397
4       984
Name: staff, dtype: Int64

In [12]:
print("Distribution of notes over staves for all pieces with more than two staves\n")
for group, df in all_notes.groupby(level=[0,1]):
    if (df.staff > 2).any():
        print(group)
        print(df.staff.value_counts().to_dict())

Distribution of notes over staves for all pieces with more than two staves

('grieg_lyrical_pieces', 'op43n06')
{2: 769, 3: 422, 1: 180}
('liszt_pelerinage', '161.04_Sonetto_47_del_Petrarca')
{1: 1076, 2: 628, 3: 42, 4: 29}
('liszt_pelerinage', '161.07_Apres_une_lecture_du_Dante')
{1: 6638, 2: 5181, 3: 50}
('liszt_pelerinage', '162.01_Gondoliera')
{3: 1745, 4: 955}
('medtner_tales', 'op34n03')
{1: 1219, 2: 816, 3: 89}
('medtner_tales', 'op35n04')
{1: 1678, 2: 1632, 3: 49}


In [13]:
all_notes[all_notes.staff > 2].groupby(level=[0,1]).staff.value_counts()

corpus                fname                              staff
grieg_lyrical_pieces  op43n06                            3         422
liszt_pelerinage      161.04_Sonetto_47_del_Petrarca     3          42
                                                         4          29
                      161.07_Apres_une_lecture_du_Dante  3          50
                      162.01_Gondoliera                  3        1745
                                                         4         955
medtner_tales         op34n03                            3          89
                      op35n04                            3          49
Name: staff, dtype: int64

# Tokens

All symbols, independent of the local key (the mode of which changes their semantics).

In [14]:
for group, dict_of_dfs in annotated.iter_facet("expanded", concatenate=True):
    for tuple_of_all_id_tuples, all_annotations in dict_of_dfs.items():
        pass
all_annotations.sample(20)

quarterbeats  \
corpus                    fname                             interval                                    
liszt_pelerinage          160.01_Chapelle_de_Guillaume_Tell [228.75, 229.0)                     915/4   
beethoven_piano_sonatas   19-2                              [8.5, 9.0)                           17/2   
tchaikovsky_seasons       op37a01                           [170.5, 171.0)                      341/2   
chopin_mazurkas           BI77-2op17-2                      [79.0, 79.5)                           79   
beethoven_piano_sonatas   23-3                              [483.0, 484.0)                        483   
medtner_tales             op48n01                           [409.0, 409.5)                        409   
                          op42n03                           [12.0, 13.5)                           12   
grieg_lyrical_pieces      op57n02                           [300.0, 304.0)                        300   
medtner_tales             op34n03                           [188.5, 189.0)                      377/2   
beethoven_piano_sonatas   32-2                              [118.25, 118.58333333333333)        473/4   
debussy_suite_bergamasque l075-02_suite_menuet              [169.5, 170.0)                      339/2   
chopin_mazurkas           BI126-1op41-4                     [124.0, 128.0)                        124   
beethoven_piano_sonatas   01-4                              [118.0, 119.0)                        118   
grieg_lyrical_pieces      op57n01                           [229.0, 230.0)                        229   
beethoven_piano_sonatas   18-4                              [490.0, 491.5)                        490   
chopin_mazurkas           BI163op67-4                       [120.0, 121.0)                        120   
grieg_lyrical_pieces      op71n02                           [69.0, 70.0)                           69   
medtner_tales             op48n01                           [541.5, 542.0)                     1083/2   
dvorak_silhouettes        op08n12                           [216.5, 217.5)                      433/2   
                          op08n03                           [119.5, 120.0)                      239/2   

                                                                                         duration_qb  \
corpus                    fname                             interval                                   
liszt_pelerinage          160.01_Chapelle_de_Guillaume_Tell [228.75, 229.0)                     0.25   
beethoven_piano_sonatas   19-2                              [8.5, 9.0)                           0.5   
tchaikovsky_seasons       op37a01                           [170.5, 171.0)                       0.5   
chopin_mazurkas           BI77-2op17-2                      [79.0, 79.5)                         0.5   
beethoven_piano_sonatas   23-3                              [483.0, 484.0)                       1.0   
medtner_tales             op48n01                           [409.0, 409.5)                       0.5   
                          op42n03                           [12.0, 13.5)                         1.5   
grieg_lyrical_pieces      op57n02                           [300.0, 304.0)                       4.0   
medtner_tales             op34n03                           [188.5, 189.0)                       0.5   
beethoven_piano_sonatas   32-2                              [118.25, 118.58333333333333)    0.333333   
debussy_suite_bergamasque l075-02_suite_menuet              [169.5, 170.0)                       0.5   
chopin_mazurkas           BI126-1op41-4                     [124.0, 128.0)                       4.0   
beethoven_piano_sonatas   01-4                              [118.0, 119.0)                       1.0   
grieg_lyrical_pieces      op57n01                           [229.0, 230.0)                       1.0   
beethoven_piano_sonatas   18-4                              [490.0, 491.5)                       1.5   
chopin_maz

In [15]:
no_chord = all_annotations.root.isna()
print(f"Concatenated annotation tables contains {all_annotations.shape[0]} rows. {no_chord.sum()} of them are not chords. Their values are:")
all_annotations.label[no_chord].value_counts(dropna=False).to_dict()

Concatenated annotation tables contains 57566 rows. 677 of them are not chords. Their values are:


{'{': 640, '}': 28, '|PAC}': 6, '|HC': 2, '|PAC': 1}

In [16]:
all_chords = all_annotations[~no_chord]
print(f"Corpus contains {all_chords.shape[0]} tokens and {len(all_chords.chord.unique())} types over {len(tuple_of_all_id_tuples)} documents.")

Corpus contains 56889 tokens and 3115 types over 264 documents.


In [17]:
#from ms3 import write_tsv
#write_tsv(all_annotations[all_annotations.pedalend.notna()], './issues/pedalpoints.tsv', pre_process=False)

# Unigrams
For computing unigram statistics, the tokens need to be grouped by their occurrence within a major or a minor key because this changes their meaning. To that aim, the annotated corpus needs to be sliced into contiguous localkey segments which are then grouped into a major (`is_minor=False`) and a minor group.

In [18]:
localkey_slices = LocalKeySlicer().process_data(annotated)

In [19]:
mode_slices = ModeGrouper().process_data(localkey_slices)

## Whole dataset

In [20]:
unigrams = ChordSymbolUnigrams(once_per_group=True).process_data(mode_slices)

In [21]:
unigrams.get()

localkey_is_minor  chord        
False              I                5229
                   V7               2597
                   V                2176
                   I6               1757
                   IV                981
                                    ... 
True               I(#6)               1
                   V(642)              1
                   I(94)               1
                   It/iv               1
                   #viio2(+2)/ii       1
Name: count, Length: 3715, dtype: int64

In [22]:
modes = {True: 'MINOR', False: 'MAJOR'}
for (is_minor,), ugs in unigrams.iter():
    print(f"{modes[is_minor]} UNIGRAMS\n{ugs.shape[0]} types, {ugs.sum()} tokens")
    print(ugs.head(20).to_string())

MAJOR UNIGRAMS
2004 types, 35610 tokens
chord
I        5229
V7       2597
V        2176
I6       1757
IV        981
V(64)     722
V43       670
V2        647
V6        638
vi        610
ii        570
ii6       548
V65       541
IV6       469
I64       445
V7(9)     363
viio6     347
V7/V      324
ii7       278
V7/IV     236
MINOR UNIGRAMS
1711 types, 21279 tokens
chord
i          2703
V          1413
V7         1173
i6          914
iv          489
V(64)       458
V6          354
VI          321
iv6         314
V43         303
i64         243
III         233
V65         222
V2          213
#viio43     209
I           208
v           170
bII         161
V7/III      158
#viio7      156


## Per corpus

In [23]:
corpus_wise_unigrams = Pipeline([CorpusGrouper(), ChordSymbolUnigrams(once_per_group=True)]).process_data(mode_slices)

In [24]:
corpus_wise_unigrams.get()

localkey_is_minor  corpus                   chord   
False              beethoven_piano_sonatas  I           2232
                                            V           1189
                                            I6          1125
                                            V7          1041
                                            V6           468
                                                        ... 
True               tchaikovsky_seasons      i(4)/v         1
                                            i/v            1
                                            ii64           1
                                            v6/iv          1
                                            ii%65(2)       1
Name: count, Length: 6705, dtype: int64

In [25]:
for (is_minor, corpus_name), ugs in corpus_wise_unigrams.iter():
    print(f"{corpus_name} {modes[is_minor]} unigrams ({ugs.shape[0]} types, {ugs.sum()} tokens)")
    print(ugs.head(5).to_string())

beethoven_piano_sonatas MAJOR unigrams (842 types, 15275 tokens)
chord
I     2232
V     1189
I6    1125
V7    1041
V6     468
chopin_mazurkas MAJOR unigrams (451 types, 5256 tokens)
chord
I       949
V7      659
V       252
IV      189
V7/V    128
debussy_suite_bergamasque MAJOR unigrams (106 types, 363 tokens)
chord
I         28
V7        22
iii       19
V7(+2)    12
IV6       12
dvorak_silhouettes MAJOR unigrams (140 types, 1067 tokens)
chord
I     254
I6    103
V7     89
V      79
IV     46
grieg_lyrical_pieces MAJOR unigrams (659 types, 4882 tokens)
chord
I     566
V7    264
V     224
IV    127
vi    116
liszt_pelerinage MAJOR unigrams (552 types, 3455 tokens)
chord
I     501
V7    250
V     155
I6    109
IV    108
medtner_tales MAJOR unigrams (636 types, 2864 tokens)
chord
I     263
V     139
V7    122
I6     83
IV     61
schumann_kinderszenen MAJOR unigrams (114 types, 714 tokens)
chord
I      106
I6      69
V       65
V7      50
V43     30
tchaikovsky_seasons MAJOR unigrams (194

## Per piece

In [26]:
piece_wise_unigrams = Pipeline([PieceGrouper(), ChordSymbolUnigrams(once_per_group=True)]).process_data(mode_slices)

In [27]:
piece_wise_unigrams.get()

localkey_is_minor  fname    chord 
False              01-1     V7        11
                            I6         8
                            I          7
                            ii6        5
                            V          5
                                      ..
True               op71n07  ii%65      3
                            VIM43      2
                            ii%7       2
                            V(964)     2
                            #viio2     1
Name: count, Length: 16090, dtype: int64

# Bigrams

## Whole dataset

In [28]:
bigrams = ChordSymbolBigrams(once_per_group=True).process_data(mode_slices)

In [29]:
bigrams.get()

localkey_is_minor  from  to      
False              V7    I           1254
                   V     I            586
                   I     V            443
                         V7           406
                   V2    I6           344
                                     ... 
True               V7    bVI            1
                         VIM7(+2)       1
                         VIM7           1
                         VI(11)         1
                   vo7   i              1
Name: count, Length: 15149, dtype: int64

In [30]:
modes = {True: 'MINOR', False: 'MAJOR'}
for (is_minor,), ugs in bigrams.iter():
    print(f"{modes[is_minor]} BIGRAMS\n{ugs.shape[0]} transition types, {ugs.sum()} tokens")
    print(ugs.head(20).to_string())

MAJOR BIGRAMS
8539 transition types, 34721 tokens
from   to 
V7     I      1254
V      I       586
I      V       443
       V7      406
V2     I6      344
V(64)  V7      310
I      I6      294
I6     I       292
V65    I       266
I      IV      245
V      V7      233
V43    I       218
I      V43     210
IV     I       164
V6     I       164
I      V6      161
V      I6      147
V(64)  V       142
I6     IV      139
I      V2      132
MINOR BIGRAMS
6610 transition types, 20690 tokens
from   to   
V7     i        573
V      i        417
i      V        209
       V7       192
       i6       172
V(64)  V7       149
i6     i        145
V      V7       142
V(64)  V        138
V43    i        133
V65    i        129
V6     i        118
i      V6       104
V2     i6        94
i      iv        89
       VI        80
       V43       75
V7(6)  V7        65
V7     V(64)     61
i(9)   i         57


## Per corpus

In [31]:
corpus_wise_bigrams = Pipeline([CorpusGrouper(), ChordSymbolBigrams(once_per_group=True)]).process_data(mode_slices)

In [32]:
corpus_wise_bigrams.get()

localkey_is_minor  corpus                   from    to      
False              beethoven_piano_sonatas  V7      I           532
                                            V       I           323
                                            V2      I6          238
                                            I       V           230
                                            V(64)   V7          213
                                                               ... 
True               tchaikovsky_seasons      i(4)/v  i/v           1
                                            V2/bII  ii%65(2)      1
                                            V7(2)   V7            1
                                            ii%43   V             1
                                            ii%65   ii%7(4)       1
Name: count, Length: 19367, dtype: int64

In [33]:
for (is_minor, corpus_name), ugs in corpus_wise_bigrams.iter():
    print(f"{corpus_name} {modes[is_minor]} bigrams ({ugs.shape[0]} transition types, {ugs.sum()} tokens)")
    print(ugs.head(5).to_string())

beethoven_piano_sonatas MAJOR bigrams (3509 transition types, 15028 tokens)
from   to
V7     I     532
V      I     323
V2     I6    238
I      V     230
V(64)  V7    213
chopin_mazurkas MAJOR bigrams (1394 transition types, 5135 tokens)
from  to
V7    I     338
I     IV     96
      V7     93
      V      70
IV    I      67
debussy_suite_bergamasque MAJOR bigrams (245 transition types, 354 tokens)
from  to   
I     iii      8
V7    I        7
iii6  V7       5
iii   bIII6    5
      V64      4
dvorak_silhouettes MAJOR bigrams (347 transition types, 1013 tokens)
from  to
V7    I     64
I     V7    45
V     I     37
V2    I6    29
I     I6    21
grieg_lyrical_pieces MAJOR bigrams (1735 transition types, 4755 tokens)
from   to
V7     I     102
I      V      39
V      I      37
V7(9)  I      31
I      V7     30
liszt_pelerinage MAJOR bigrams (1471 transition types, 3323 tokens)
from  to
V7    I     97
V     I     47
I     V     44
      V7    42
I(9)  I     41
medtner_tales MAJOR bigrams (

## Per piece

In [34]:
piece_wise_bigrams = Pipeline([PieceGrouper(), ChordSymbolBigrams(once_per_group=True)]).process_data(mode_slices)

(('beethoven_piano_sonatas', '23-2', Interval(190.0, 194.0, closed='left')),): DataFrame has only one row, cannot compute bigram.
Group '(True, '23-2')' will be missing from the processed data.


In [35]:
piece_wise_bigrams.get()

localkey_is_minor  fname    from     to    
False              01-1     V(64)    V7        4
                            viio7/V  V(64)     3
                            ii6(2)   ii6       3
                            V65/V    V         3
                            V7(+b9)  V7        3
                                              ..
True               op71n07  V(964)   ii%65     2
                            VIM43    ii%7      2
                            ii%65    VIM43     2
                            ii%7     V(964)    2
                            ii%65    #viio2    1
Name: count, Length: 27544, dtype: int64